# Решение задачи оценки ожидаемых выплат по полису КАСКО методом градиентного бустинга

## Предобработка данных

### * Домашнее задание: Многоклассовая классификация

В текущем домашнем задание предлагается взглянуть на задачу моделирования количества страховых случаев как на задачу многоклассовой классификации.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix




In [2]:
#!pip install hyperopt

In [3]:
from hyperopt import hp, tpe, space_eval
from hyperopt.fmin import fmin

In [4]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import GridSearchCV

In [5]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [6]:
# Загрузка набора данных в pandas DataFrame
df = pd.read_csv('D:/AI/Machine learning/freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)


In [7]:
df.reset_index(drop=True, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

In [9]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Предобработайте данные

In [10]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [11]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

In [12]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [13]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [14]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [15]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 29 columns):
Exposure                           115155 non-null float64
LicAge                             115155 non-null int64
RecordBeg                          115155 non-null object
RecordEnd                          59455 non-null object
Gender                             115155 non-null int64
MariStat                           115155 non-null int64
DrivAge                            115155 non-null int64
HasKmLimit                         115155 non-null int64
BonusMalus                         115155 non-null int64
ClaimAmount                        115155 non-null float64
ClaimInd                           115155 non-null int64
ClaimNbResp                        115155 non-null float64
ClaimNbNonResp                     115155 non-null float64
ClaimNbParking                     115155 non-null float64
ClaimNbFireTheft                   115155 non-null float64
ClaimNbWindscreen    

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [17]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [18]:
df['DrivAgeSq'] = df.loc[:, 'DrivAge'].apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [19]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

Для моделирования среднего убытка можем рассчитать его как отношение величины убытков к их частоте.

In [20]:
df['AvgClaim'] = 0
for d in range(len(df['ClaimsCount'])):
    if df.loc[d, 'ClaimsCount'] > 0:
        df.loc[d, 'AvgClaim'] = df.loc[d, 'ClaimAmount']/df.loc[d, 'ClaimsCount']

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 25 columns):
Exposure                           115155 non-null float64
LicAge                             115155 non-null int64
Gender                             115155 non-null int64
MariStat                           115155 non-null int64
DrivAge                            115155 non-null int64
HasKmLimit                         115155 non-null int64
BonusMalus                         115155 non-null int64
ClaimAmount                        115155 non-null float64
ClaimInd                           115155 non-null int64
OutUseNb                           115155 non-null float64
RiskArea                           115155 non-null float64
VehUsage_Private                   115155 non-null uint8
VehUsage_Private+trip to office    115155 non-null uint8
VehUsage_Professional              115155 non-null uint8
VehUsage_Professional run          115155 non-null uint8
SocioCateg_CSP1           

XGBoost для многоклассовой классификации принимает на вход значения меток классов в виде `[0, num_classes]`. Заменим значение 11 на 10.

In [22]:
df.ClaimsCount.unique()

array([ 0.,  4.,  2.,  1.,  3.,  5.,  6.,  7.,  8.,  9., 11.])

In [23]:
df.ClaimsCount.replace(to_replace=11, value=10, inplace=True)
df.ClaimsCount.unique()

array([ 0.,  4.,  2.,  1.,  3.,  5.,  6.,  7.,  8.,  9., 10.])

Посмотрим, сколько полисов соответствуют каждому из значений `ClaimsCount`, используя метод `groupby`. Для полученных значений также посчитаем нормированную частоту.

In [24]:
fc = df.groupby('ClaimsCount')['Exposure'].sum()
FreqCount = pd.DataFrame(data=fc.values, index=fc.index, columns=['ExposureYears'])
FreqCount

,ExposureYears
ClaimsCount,
0.0,44362.363
1.0,2023.495
2.0,2134.884
3.0,1382.924
4.0,641.562
5.0,245.889
6.0,73.609
7.0,13.378
8.0,2.882


Couldn't understand what normalized frequency means. Forming some data on number of exposure years to number of claims and their share in total Exposure correspondingly.

In [25]:
#number of policies for each claimsnumber catergory
FreqCount['PolicyNumber'] = df.ClaimsCount.value_counts()

In [26]:
FreqCount

,ExposureYears,PolicyNumber
ClaimsCount,,
0.0,44362.363,104286
1.0,2023.495,3339
2.0,2134.884,3529
3.0,1382.924,2310
4.0,641.562,1101
5.0,245.889,428
6.0,73.609,127
7.0,13.378,26
8.0,2.882,6


In [27]:
#number of years of exosure per each claim for each claimsnumber catergory
FreqCount['ClaimsPerYear'] = FreqCount.iloc[1:,0]/FreqCount.index[1:]

In [28]:
#in the first row 0 is denominator, so the value from inf we change to 0
FreqCount.loc[0, 'ClaimsPerYear'] = 0

In [29]:
#share of each category in exposure
FreqCount['ExposureShare'] = FreqCount['ExposureYears']/FreqCount['ExposureYears'].sum()

In [30]:
#share of each category in policies
FreqCount['PolicyShare'] = FreqCount['PolicyNumber']/FreqCount['PolicyNumber'].sum()

In [31]:
FreqCount['weight'] = FreqCount['ExposureShare']*FreqCount['PolicyShare']

In [32]:
FreqCount.reset_index(inplace=True)

In [33]:
FreqCount

,ClaimsCount,ExposureYears,PolicyNumber,ClaimsPerYear,ExposureShare,PolicyShare,weight
0,0.0,44362.363,104286,0.000000,0.871865,0.905614,7.895732e-01
1,1.0,2023.495,3339,2023.495000,0.039768,0.028996,1.153109e-03
2,2.0,2134.884,3529,1067.442000,0.041957,0.030646,1.285813e-03
3,3.0,1382.924,2310,460.974667,0.027179,0.020060,5.452078e-04
4,4.0,641.562,1101,160.390500,0.012609,0.009561,1.205529e-04
5,5.0,245.889,428,49.177800,0.004833,0.003717,1.796117e-05
6,6.0,73.609,127,12.268167,0.001447,0.001103,1.595462e-06
7,7.0,13.378,26,1.911143,0.000263,0.000226,5.936306e-08
8,8.0,2.882,6,0.360250,0.000057,0.000052,2.951189e-09
9,9.0,0.749,2,0.083222,0.000015,0.000017,2.556605e-10


Заметим, что в данном случае присутствует проблема несбалансированности классов. Поэтому, для того, чтобы по возможности избежать ее, воспользуемся взвешиванием наблюдений для обучения модели. Для этого в исходном наборе данных создадим столбец `weight`. Присвоим ему некоторые значения, например, можно задать `0.05` для значений `ClaimsCount` 0, а для остальных - 1 (Для этого можем использовать функцию `np.where`). Также можно попробовать какой-либо другой способ задания весов, приведенный пример не гарантирует хороших результатов.

i'll try undersampling after checking unbalanced model. may be using claimscount-dependent weights

In [34]:
df = pd.merge(df, FreqCount.loc[:,['ClaimsCount','weight']], on='ClaimsCount', how='left')

In [35]:
df.tail(10)

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,OutUseNb,...,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq,ClaimsCount,AvgClaim,weight
115145,0.500,130,1,0,34,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,1156,0.0,0.000000,0.789573
115146,0.092,136,1,0,35,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,1225,0.0,0.000000,0.789573
115147,0.157,352,0,0,53,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,2809,0.0,0.000000,0.789573
115148,0.422,353,0,0,53,0,50,1117.886103,1,0.0,...,0,0,0,1,0,0,2809,1.0,1117.886103,0.001153
115149,0.420,358,0,0,53,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,2809,0.0,0.000000,0.789573
115150,0.423,238,0,0,39,0,50,0.000000,0,4.0,...,0,0,0,1,0,0,1521,0.0,0.000000,0.789573
115151,1.000,408,1,0,54,0,50,2764.169184,1,0.0,...,0,0,0,1,0,0,2916,2.0,1382.084592,0.001286
115152,0.805,211,0,0,35,0,54,0.000000,0,0.0,...,0,0,0,1,0,0,1225,0.0,0.000000,0.789573
115153,0.538,356,0,0,52,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,2704,0.0,0.000000,0.789573
115154,0.461,362,0,0,52,0,50,0.000000,0,0.0,...,0,0,0,1,0,0,2704,0.0,0.000000,0.789573


In [37]:
df.columns

Index(['Exposure', 'LicAge', 'Gender', 'MariStat', 'DrivAge', 'HasKmLimit',
       'BonusMalus', 'ClaimAmount', 'ClaimInd', 'OutUseNb', 'RiskArea',
       'VehUsage_Private', 'VehUsage_Private+trip to office',
       'VehUsage_Professional', 'VehUsage_Professional run', 'SocioCateg_CSP1',
       'SocioCateg_CSP2', 'SocioCateg_CSP3', 'SocioCateg_CSP4',
       'SocioCateg_CSP5', 'SocioCateg_CSP6', 'SocioCateg_CSP7', 'DrivAgeSq',
       'ClaimsCount', 'AvgClaim', 'weight'],
      dtype='object')

Разобьем имеющийся набор данных на обучающую, валидационную и тестовую выборки в отношениях 70%/15%/15% соответственно. Зададим зерно для случайного разбиения равным 10.

In [38]:
features = ['Exposure', 'LicAge', 'Gender', 'MariStat', 'DrivAge', 'HasKmLimit',
       'BonusMalus', 'OutUseNb', 'RiskArea', 'VehUsage_Private', 'VehUsage_Private+trip to office',
       'VehUsage_Professional', 'VehUsage_Professional run', 'SocioCateg_CSP1',
       'SocioCateg_CSP2', 'SocioCateg_CSP3', 'SocioCateg_CSP4',
       'SocioCateg_CSP5', 'SocioCateg_CSP6', 'SocioCateg_CSP7', 'DrivAgeSq']
target = ['ClaimsCount']

In [40]:
x_train, x_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.3, random_state=10)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=10)

Далее, создадим объекты `DMatrix` для обучающей, валидационной и тестовой выборок. Для обучающей выборки также укажем параметр `weight` равным полученному ранее столбцу весов. Данный столбец также нужно исключить из объекта передаваемого в параметр `data`.

In [41]:
xgb_train = xgb.DMatrix((x_train), (y_train+1))
xgb_valid = xgb.DMatrix((x_valid), (y_valid+1))
xgb_test = xgb.DMatrix((x_test), (y_test+1))

Для оптимизации гиперпараметров можно воспользоваться различными методами.

In [44]:
# Зададим функцию Deviance для распределения Пуассона

def xgb_eval_dev_poisson(yhat, dtrain):
    """
    Function for Poisson Deviance evaluation

    :param yhat: np.ndarray object with predictions
    :param dtrain: xgb.DMatrix object with target variable
    :return: (str, float), tuple with metrics name and its value
    """
    y = dtrain.get_label()
    return 'dev_poisson', 2 * np.sum( y*np.log(y/yhat) - (y-yhat) )

In [45]:
# Определим функцию для оптимизации гиперпараметров алгоритмом TPE

def objective(params):
    """
    Objective function for hyperopt. Optimizing mean cross-validation error with XGBoost.

    :param params: dict object passed to hyperopt fmin() function
    :return: float, mean cross-validation error for XGBoost utilizing params
    """
    params['max_depth'] = int(params['max_depth'])
    n_b_r = int(params.pop('num_boost_round'))
    data = params.pop('data')
    feval = params.pop('feval')
    nfold = params.pop('nfold')
    e_s_r = params.pop('early_stopping_rounds')
    maximize = params.pop('maximize')
    cv_result = xgb.cv(params, data, num_boost_round=n_b_r, nfold=nfold, seed=0, maximize=maximize,
                       feval=feval, early_stopping_rounds=e_s_r)
    name, _ = feval(data.get_label(), data)
    score = cv_result['test-{}-mean'.format(name)][-1:].values[0]
    return score

In [47]:
# Определим границы, в которых будем искать гиперпараметры

space_freq = {'data': xgb_train,
              'objective': 'count:poisson',
              'feval': xgb_eval_dev_poisson,
              'maximize': False,
              'nfold': 5,
              'early_stopping_rounds': 20,
              'num_boost_round': 300,  # hp.choice('num_boost_round', [50, 300, 500])
              'max_depth': hp.choice('max_depth', [5, 8, 10, 12, 15]),
              'min_child_weight': hp.uniform('min_child_weight', 0, 50),
              'subsample': hp.uniform('subsample', 0.5, 1),
              'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
              'alpha': hp.uniform('alpha', 0, 1),
              'lambda': hp.uniform('lambda', 0, 1),
              'eta': hp.uniform('eta', 0.01, 1),
              }

In [48]:
# Оптимизация (количество итераций снижено для ускорения работы)

best = fmin(fn=objective, space=space_freq, algo=tpe.suggest, max_evals=50)

100%|███████████████████████████████| 50/50 [36:17<00:00, 43.54s/trial, best loss: 4630.7526368]


In [49]:
# Оптимальные гиперпараметры 

best_params = space_eval(space_freq, best)
best_params

{'alpha': 0.995722589161069,
 'colsample_bytree': 0.7405690357389305,
 'data': <xgboost.core.DMatrix at 0x195721910f0>,
 'early_stopping_rounds': 20,
 'eta': 0.06488854787559624,
 'feval': <function __main__.xgb_eval_dev_poisson(yhat, dtrain)>,
 'lambda': 0.7494711513476202,
 'max_depth': 5,
 'maximize': False,
 'min_child_weight': 31.002714694253594,
 'nfold': 5,
 'num_boost_round': 300,
 'objective': 'count:poisson',
 'subsample': 0.9412728856065011}

Далее обучим нашу модель с оптимальными параметрами

In [50]:
def train_xgb_best_params(params, dtrain, evals, early_stopping_rounds, evals_result=None, verbose_eval=None):
    """
    Function to train XGBoost estimator from set of parameters, passed from hyperopt.

    :param params: dict, hyperparameters from hyperopt space_eval() function
    :param dtrain: xgb.DMatrix object, to train model on
    :param evals: list of pairs (DMatrix, str). Same from xgb.train().
    :param early_stopping_rounds: int. Same from xgb.train().
    :param evals_result: dict. Same from xgb.train().
    :param verbose_eval: bool or int. Same from xgb.train().
    :return: xgb.Booster object, trained model
    """
    par = params.copy()
    for label in ['nfold', 'data', 'early_stopping_rounds']:
        del par[label]
    n_b_r = int(par.pop('num_boost_round'))
    maximize = par.pop('maximize')
    feval = par.pop('feval')
    return xgb.train(params=par, dtrain=dtrain, num_boost_round=n_b_r, evals=evals, feval=feval, maximize=maximize,
                     early_stopping_rounds=early_stopping_rounds, evals_result=evals_result, verbose_eval=verbose_eval)

In [52]:
progress = dict()
xgb_claimcount = train_xgb_best_params(best_params,
                                     xgb_train, [(xgb_train, "train"),(xgb_valid, "valid")],
                                     early_stopping_rounds=20, evals_result=progress, verbose_eval=False)

Посчитаем метрики accuracy и f1 на наших наборах данных, также можем визуализировать confusion matrix, например, с помощью `plt.imshow()`. Можно использовать предложенный ниже код.

In [54]:
dfsets = [{'set': 'train', 'dmat': xgb_train, 'target': y_train},
          {'set': 'valid', 'dmat': xgb_valid, 'target': y_valid},
          {'set': 'test', 'dmat': xgb_test, 'target': y_test}]


In [64]:
for dfset in dfsets:
    class_preds = xgb_claimcount.predict(dfset['dmat'])# Посчитаем предсказанные значения
    print(class_preds)
    print('F1 Score on ' + str(dfset['set'])+':', f1_score(dfset['target'], class_preds, average='micro')) # Посчитаем F1 Score
    print('F1 Score on ' + str(dfset['set'])+':', accuracy_score(dfset['target'], class_preds)) # Посчитаем F1 Score

[1.2013105 1.2893329 1.3850726 ... 1.4262367 1.2966355 1.6062814]


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [0]:
plt.subplots(1,3, figsize=(15,3))
for i in range(len(dfsets)):
    confmatrix = confusion_matrix(dfsets[i]['target'], xgb_multiclass.predict(dfsets[i]['dmat']))
    plt.subplot(1,3,i+1)
    plt.imshow(confmatrix, cmap='Greys')
    plt.colorbar()
    plt.ylabel('True')
    plt.xlabel('Predicted')
plt.show()

Как вы оцениваете качество построенной модели? Какие проблемы могут здесь присутствовать? Как можно улучшить результат?

Об особенностях сохранения моделей:

In [0]:
xgb_avgclaim.save_model('/avg_claim.model')

In [0]:
xgb_avgclaim.feature_names

['LicAge',
 'Gender',
 'MariStat',
 'DrivAge',
 'HasKmLimit',
 'BonusMalus',
 'OutUseNb',
 'RiskArea',
 'VehUsage_Private',
 'VehUsage_Private+trip to office',
 'VehUsage_Professional',
 'VehUsage_Professional run',
 'SocioCateg_CSP1',
 'SocioCateg_CSP2',
 'SocioCateg_CSP3',
 'SocioCateg_CSP4',
 'SocioCateg_CSP5',
 'SocioCateg_CSP6',
 'SocioCateg_CSP7',
 'DrivAgeSq']

In [0]:
xgb_avgclaim.feature_types

['int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'float',
 'float',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int',
 'int']

In [0]:
model =xgb.Booster()
model.load_model('/avg_claim.model')

In [0]:
print(model.feature_names)
type(model.feature_names)

None


NoneType

In [0]:
model.feature_types

AttributeError: ignored